In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
from src.subset_classifier import *
import pandas as pd
import sqlite3
import sqlalchemy 
import json
from datasets import load_dataset, load_metric
from collections import Counter
from pydantic.dataclasses import dataclass
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification
import uuid
import numpy as np

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# DB_PATH = "./subset_selection/sst_results.db"
# df = get_df_from_db(DB_PATH)

In [4]:
# read df from pickle file
df = pd.read_pickle("./results/sst_results_df.pkl")

In [5]:
subset_unique_counts = get_subset_unique_counts(df); print(subset_unique_counts)

Counter({96: 3164, 93: 1815, 94: 1793, 95: 876, 92: 661, 97: 262})


In [6]:
optimal_subset_data_indices = get_optimal_subset_data_indices(df)

In [7]:
config = OptimalSubsetClassifierConfig(max_length=66, debug=False, model_name="albert-base-v2", batch_size=8, max_steps=20000, )

In [8]:
train_ds, valid_ds, test_ds, debug_ds = create_train_valid_test_debug_ds(optimal_subset_data_indices, config)

print(len(train_ds), len(valid_ds), len(test_ds), len(debug_ds))

100%|██████████| 3/3 [00:00<00:00, 226.97it/s]
INFO:src.subset_classifier:dataset      num positive examples    num negative examples
---------  -----------------------  -----------------------
train                           77                      723
valid                            9                       91
test                            10                       90


800 100 100 12


In [9]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
config.debug = True; print(config)
training_args = TrainingArguments(
            output_dir="./subset_classifier_results",
            max_steps=config.max_steps if not config.debug else 300,
            evaluation_strategy="steps",
            report_to="wandb" if not config.debug else None,
            run_name=f"subset_classifier_result_{uuid.uuid4().hex}",
            eval_steps=300,
            learning_rate=1e-5,
            adam_epsilon=1e-6,
            warmup_ratio=0.1,
            weight_decay=0.01,
        )

OptimalSubsetClassifierConfig(max_length=66, debug=True, model_name='albert-base-v2', batch_size=8, max_steps=20000)


In [11]:
training_args.run_name, training_args.max_steps

('subset_classifier_result_5c35548630b845138fcb717220716040', 300)

In [12]:
training_args.report_to

['wandb']

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(config.model_name, num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds if not config.debug else debug_ds,
    eval_dataset=valid_ds if not config.debug else debug_ds,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [14]:
trainer.train()

/Users/garylai/Dev/subset-active-learning/.venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12
  Num Epochs = 150
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 300
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jjoozzz. Use `wandb login --relogin` to force relogin


  2%|▏         | 6/300 [00:19<14:40,  2.99s/it]

KeyboardInterrupt: 